# Reimplementation of original kBET for sc_04 cell type 2
- run with R(kbet)

In [1]:
#truncated normal distribution distribution function
ptnorm <- function(x, mu, sd, a=0, b=1, alpha = 0.05, verbose = FALSE){
  #this is the cumulative density of the truncated normal distribution
  #x ~ N(mu, sd^2), but we condition on a <= x <= b
  if (!is.na(x)){


    if (a > b) {
      warning("Lower and upper bound are interchanged.")
      tmp <- a
      a <- b
      b <- tmp
    }

    if (sd <= 0 || is.na(sd)) {
      if (verbose) {
        warning("Standard deviation must be positive.")
      }
      if (alpha <= 0) {
        stop("False positive rate alpha must be positive.")
      }
      sd <- alpha
    }
    if (x < a || x > b) {
      warning("x out of bounds.")
      cdf <- as.numeric(x > a)
    } else {
      alp <- pnorm((a - mu) / sd)
      bet <- pnorm((b - mu) / sd)
      zet <- pnorm((x - mu) / sd)
      cdf <- (zet - alp) / (bet - alp)
    }
    cdf
  } else {
    return(NA)
  }
}

In [2]:
# residual score function of kBET
residual_score_batch <- function(knn.set, class.freq, batch) {
    # knn.set: indices of nearest neighbors
    # class.freq: global batch proportions
    # batch: batch labels

    # return NA if all values of neighborhood are NA (which may arise from subsampling a knn-graph)
    if (all(is.na(knn.set))) {
        return(NA)
    }
    else{
        # extracts the batch labels of all neighbors (excluding NA)
        # computes local batch frequencies (observed)
        freq.env <- table(batch[knn.set[!is.na(knn.set)]]) / length(!is.na(knn.set))
        
        # create zero vector with length of batches and initialize it with freqs of batches
        full.classes <- rep(0, length(class.freq$class))
        full.classes[class.freq$class %in% names(freq.env)] <- freq.env
        
        # global batch props (expected)
        exp.freqs <- class.freq$freq

        # compute chi-square test statistics
        ## sum((observed - expected)^2 / expected)
        chi_sq_statistic <- sum((full.classes - exp.freqs)^2 / exp.freqs)
        
        return(chi_sq_statistic)
    }
}

In [3]:
# core function of kBET; called for each sample
chi_batch_test <- function(knn.set, class.freq, batch, df) {
    # knn.set: indices of nearest neighbors
    # class.freq: global batch proportions
    # batch: batch labels
    # df: degrees of freedom

    # return NA if all values of neighborhood are NA (which may arise from subsampling a knn-graph)
    if (all(is.na(knn.set))) {
        return(NA)
    }
    else {
        # extracts the batch labels of all neighbors (excluding NA)
        # computes local batch counts (observed)
        freq.env <- table(batch[knn.set[!is.na(knn.set)]])

        # create zero vector with length of batches and initialize it with counts of batches
        full.classes <- rep(0, length(class.freq$class))
        full.classes[class.freq$class %in% names(freq.env)] <- freq.env

        # global batch counts (for this sample)
        exp.freqs <- class.freq$freq * length(knn.set)

        # compute chi-square test statistics
        ## sum((observed - expected)^2 / expected)
        chi.sq.value <- sum((full.classes - exp.freqs)^2 / exp.freqs)

        # calculate p-value
        result <- 1 - pchisq(chi.sq.value, df)

        # I actually would like to know when 'NA' arises.
        if (is.na(result)) {
            return(NA)
        }
        else {
           return(result)
        }
    }
}

In [4]:
# df: dataset (rows: cells, columns: genes)
# batch: batch labels for each cell
# k0 = NULL: # neighbors to test on
# knn = NULL: n_cells x k0 matrix of kNN indices (optional)
# testSize = NULL: # data points to test (default is 10% of entire dataset, but at least 25)
# do.pca = TRUE: perform PCA before kNN search
# dim.pca = 50: if do.pca=TRUE then # dims
# heuristic = TRUE: compute an optimal neighborhood size k
# n_repeat = 100: create statistics on batch estimates running test on n_repeat subsets
# alpha = 0.05: significance level
# addTest = FALSE: perform Likelihood Ratio Test approximation to the multinimial test and
#                a multinomial exact test (if appropriate)
# verbose = FALSE: display stages of current computation
# plot = TRUE: if stats > 10 then a boxplot of the resulting rejection rates is created
#adapt = TRUE: in some cases a number of cells do not contribute to any neighborhood and 
#              this may cause an imbalance in observed and expected batch label frequencies.
#              Frequencies will be adapted if adapt=TRUE.


In [5]:
# source("kBET-utils.R")

In [6]:
library(ggplot2)

In [7]:
sc_dir <- "2/original_kbet/"
sc_dir

[1] "2/original_kbet/"

In [8]:
df <- as.matrix(read.csv(paste0(sc_dir, "df.csv"), header=FALSE))
batch <- factor(read.csv(paste0(sc_dir, "batch.csv"), header = FALSE)[[1]])
k0 <- 70
knn <- as.matrix(read.csv(paste0(sc_dir, "knn.csv"), header = FALSE))
testSize = NULL
do.pca = FALSE
dim.pca = 50
heuristic = FALSE
n_repeat = 100
alpha = 0.05
addTest = FALSE
verbose = TRUE
plot = TRUE
adapt = FALSE

dim(df)
length(batch)
k0
dim(knn)
testSize
do.pca
dim.pca
heuristic
n_repeat
alpha
addTest
verbose
plot
adapt

[1] 600  71

[1] 600

[1] 70

[1] 600  70

NULL

...

[1] 0.05

[1] FALSE

[1] TRUE

[1] TRUE

[1] FALSE

In [9]:
length(unique(batch))

[1] 2

In [10]:
is.factor(batch)

[1] TRUE

In [11]:
dof <- length(unique(batch)) - 1    # degrees of freedom

dof

[1] 1

In [12]:
if (is.factor(batch)) {
    batch <- droplevels(batch)
}

batch

[1] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [10] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [19] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [28] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [37] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [46] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [55] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [64] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [73] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [82] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
 [91] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[100] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[109] batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1 batch 1
[118] batch 1 batch 1 batch 1 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[127] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[136] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[145] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[154] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[163] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[172] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[181] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[190] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[199] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[208] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[217] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[226] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[235] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[244] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[253] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[262] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[271] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[280] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[289] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[298] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[307] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[316] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[325] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[334] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[343] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[352] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[361] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[370] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[379] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[388] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[397] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[406] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[415] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[424] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[433] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[442] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[451] batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2 batch 2
[460] batch 2 batch 2 ba

In [13]:
table(batch)
length(batch)

batch
batch 1 batch 2 
    120     480 

[1] 600

In [14]:
frequencies <- table(batch) / length(batch)

frequencies

batch
batch 1 batch 2 
    0.2     0.8 

In [15]:
# get 3 different permutations of the batch label
batch.shuff <- replicate(3, batch[sample.int(length(batch))])

head(batch.shuff)

batch 1,batch 2,batch 2
batch 2,batch 2,batch 2
batch 2,batch 2,batch 2
batch 2,batch 1,batch 2
batch 2,batch 2,batch 2
batch 1,batch 2,batch 2


In [16]:
dim(batch.shuff)

[1] 600   3

In [17]:
class.frequency <- data.frame(
    class = names(frequencies),
    freq = as.numeric(frequencies)
)

class.frequency

class,freq
<chr>,<dbl>
batch 1,0.2
batch 2,0.8


In [18]:
dataset <- df

dim(dataset)

[1] 600  71

In [19]:
dim.dataset <- dim(dataset)

dim.dataset

[1] 600  71

In [20]:
dim.dataset[1]
dim.dataset[2]

[1] 600

[1] 71

In [21]:
# check dimensions
if (dim.dataset[1] != length(batch) && dim.dataset[2] != length(batch)) {
    stop("Input matrix and batch information do not match. Execution halted.")
}

In [22]:
if (dim.dataset[2] == length(batch) && dim.dataset[1] != length(batch)) {
    if (verbose) {
      cat('Input matrix has samples as columns. kBET needs samples as rows. Transposing...\n')
    }
    dataset <- t(dataset)
    dim.dataset <- dim(dataset)
  }

In [23]:
# check if the dataset is too small
if (dim.dataset[1] <= 10) {
    if (verbose) {
        cat("Your dataset has less than 10 samples. Abort and return NA.\n")
    }
    return(NA)
}

In [24]:
stopifnot(class(n_repeat) == 'numeric', n_repeat > 0)

In [25]:
# if k0 was set by the user and is too small & we do not operate on a knn graph, abort

# the reason is that if we want to test kBET on knn graph data integration methods,
# we usually face small numbers of nearest neighbours.
if (k0 < 10 & is.null(knn))
{
    if (verbose)
    {
        warning(
            "Your dataset has too few samples to run a heuristic.\n",
            "Return NA.\n",
            "Please assign k0 and set heuristic=FALSE."
        )
    }
    return(NA)
}

In [26]:
# set # tests
if (is.null(testSize) || (floor(testSize) < 1 || dim.dataset[1] < testSize))
{
    test.frac <- 0.10   # 10% of datapoints
    testSize <- ceiling(dim.dataset[1] * test.frac)
    
    if (testSize < 2 && dim.dataset[1] > 25)
    {
        testSize <- 25
    }

    if (verbose)
    {
        cat('Number of kBET tests is set to ')
        cat(paste0(testSize, '.\n'))
    }
}

Number of kBET tests is set to 60.


In [27]:
# result list
rejection <- list()
rejection$summary <- data.frame(
    kBET.expected = numeric(4),
    kBET.observed = numeric(4),
    kBET.signif = numeric(4)
)

rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0


In [28]:
rejection$results <- data.frame(
    tested = numeric(dim.dataset[1]),
    kBET.pvalue.test = rep(0, dim.dataset[1]),
    kBET.pvalue.null = rep(0, dim.dataset[1])
)

rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0


In [29]:
knn

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70
11,115,92,8,109,81,108,53,74,23,⋯,68,30,79,51,49,47,103,73,119,15
24,38,58,104,52,96,9,91,46,8,⋯,116,35,62,26,48,102,32,34,15,16
81,97,106,47,59,70,36,38,1,26,⋯,46,13,34,15,35,62,119,87,89,31
47,101,72,51,2,11,3,94,1,44,⋯,58,81,33,35,110,46,23,15,105,12
8,58,40,51,47,37,107,42,83,89,⋯,72,110,82,15,94,62,16,34,91,95
47,30,99,63,98,49,75,7,25,43,⋯,4,77,42,34,53,110,102,107,36,15
46,36,86,30,72,95,3,2,65,14,⋯,62,53,74,34,29,102,31,80,15,117
9,92,45,103,89,74,7,36,118,60,⋯,49,105,31,4,117,64,34,116,101,15
37,27,14,30,99,53,81,69,67,31,⋯,52,36,56,41,66,102,15,26,62,34
108,79,90,27,25,1,3,99,42,14,⋯,62,98,12,11,40,15,34,87,68,81


In [30]:
# concat knn matrix so
## 1st col comes 1st then 2nd col ... then k0-1 th col then cells itself
env <- as.vector(cbind(knn[, seq_len(k0 - 1)], seq_len(dim.dataset[1])))
env

[1]  11  24  81  47   8  47  46   9  37 108  47  71  50  13  47 105  86  84
   [19]  71  44  36 113 111  53 113 109 113   9 115 106  54 103  61 108 113  11
   [37]  38  47   5  99  53  70 106  60  81  47 120  64  94  11  64  26  36  94
   [55]  59  25  63  84 100   7  59  35 106  59  52  36  69 111  13 109  67  32
   [73]   9 113  29  46 113  59 107   9  58  14  89  29  24  37  72  98  32  66
   [91]  35  91  60  42  21 105  74  86 111  14   2  45   8   5   3  48  29  33
  [109]  97  43  17  28  30  67   9  30  19  42 118  89 316 154 433 157 563 125
  [127] 449 344 489 346 140 194 309 446 139 559 362 305 412 231 476 376 140 342
  [145] 466 481 480 137 481 489 559 469 144 574 398 308 585 205 301 340 173 334
  [163] 145 254 282 510 121 452 454 513 356 555 574 212 308 443 504 367 261 405
  [181] 138 145 318 348 491 377 480 447 342 525 434 330 488 254 309 356 504 209
  [199] 385 186 161 396 409 348 568 556 346 417 561 356 574 473 438 440 568 418
  [217] 159 450 259 563 233 152 356 338 212 488 145 174 448 539 228 428 125 400
  [235] 431 460 307 266 590 153 581 221 205 559 348 488 348 371 231 578 231 394
  [253] 423 392 496 186 515 254 381 461 356 267 556 342 423 488 526 182 179 356
  [271] 211 349 289 435 289 542 209 396 328 433 568 396 491 295 333 402 334 543
  [289] 516 315 156 551 496 466 559 261 189 575 232 463 201 344 530 368 177 341
  [307] 271 333 205 412 500 167 536 505 583 167 230 206 343 349 358 348 142 394
  [325] 571 249 213 289 583 148 590 309 220 489 342 431 356 328 574 224 145 500
  [343] 229 270 504 281 277 518 352 152 412 234 556 451 313 530 424 239 330 404
  [361] 275 372 171 139 556 291 132 458 506 308 543 405 553 405 522 201 312 309
  [379] 425 528 437 125 411 556 460 304 530 492 125 554 144 233 567 207 474 366
  [397] 532 125 328 432 568 556 213 330 494 433 451 480 348 132 448 265 556 396
  [415] 197 489 360 412 522 360 148 563 341 563 556 477 454 219 230 412 523 488
  [433] 451 472 258 167 500 432 330 264 551 474 424 351 144 595 337 352 200 218
  [451] 547 572 431 526 306 425 172 472 299 488 311 461 193 458 300 334 536 213
  [469] 348 515 205 334 401 559 516 166 199 133 454 563 338 373 143 334 125 412
  [487] 365 598 381 214 125 534 248 201 278 437 231 266 309 264 213 318 394 344
  [505] 591 473 275 152 225 410 167 430 367 559 352 563 595 487 390 309 137 474
  [523] 357 488 448 419 430 459 481 428 196 151 360 231 446 146 261 597 299 368
  [541] 270 377 136 167 339 412 555 370 255 314 211 358 360 424 281 446 518 540
  [559] 135 137 368 427 339 569 257 266 518 533 239 470 598 584 301 434 213 446
  [577] 135 236 254 395 125 230 287 567 489 213 487 418 294 250 446 349 343 553
  [595] 418 435 419 167 129 533 115  38  97 101  58  30  36  92  27  79 106   3
  [613]  29  58  94  29 111 104 111  68  58  74 100  81  63  53 111  72 113  75
  [631]  50  53  11 111  60 108  33  65 108   5  70  94  70  59   1 113 113  23
  [649]  93  83  90   8  51   2   1  65  11  37  72  58  42  67 112  74  25  81
  [667]  86  52  84 120  26  47  38  36  50  38  55  64  65  19  50  43  97  92
  [685] 112  82  74  22  19  10  81  85  87  33  87  44  79 120  60 109  25  37
  [703] 100  37  83  26 120  48  44  69   6  14 109 113  36  42   2  54  77  87
  [721] 393 348 525 126 247 254 412 159 459 298 532 556 554 158 196 193 526 340
  [739] 431 334 125 583 563 412 402 201 334 215 579 546 581 412 338 453 537 121
  [757] 333 348 330 548 464 240 230 245 438 373 489 539 563 345 450 266 128 488
  [775] 205 342 316 342 500 480 348 248 242 559 125 562 568 228 201 151 197 412
  [793] 394 556 135 474 421 217 535 353 482 405 365 155 173 394 135 321 360 174
  [811] 583 325 134 417 430 600 317 239 536 415 421 580 418 213 476 474 309 222
  [829] 125 125 567 288 309 412 407 521 375 363 521 588 394 433 394 479 567 199
  [847] 394 567 559 427 334 307 485 324 214 200 325 403 266 599 207 496 299 321
  [865] 349 242 305 286 563 418 454 313 167 268 368 556 213 556 334 161 575 328
  [883] 394 335 254 442 392 530 239 234 536 222 597 309 231 482 503 201 403 563
  [9

In [31]:
length(env)

[1] 42000

In [32]:
cf <- if (adapt && is.imbalanced) new.class.frequency else class.frequency
cf

class,freq
<chr>,<dbl>
batch 1,0.2
batch 2,0.8


In [33]:
length(batch)

[1] 600

In [34]:
# get avg p-value, p = 1 - CDF
rejection$average.pval <- 1 - pchisq(k0 * residual_score_batch(env, cf, batch), dof)
rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0


In [35]:
# initialize intermediates
kBET.expected <- numeric(n_repeat)
kBET.observed <- numeric(n_repeat)
kBET.signif <- numeric(n_repeat)

length(kBET.expected)
kBET.expected

[1] 100

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [36]:
testSize

[1] 60

In [37]:
alpha

[1] 0.05

In [38]:
# kBET; run n_repeat = 100 times
for (i in seq_len(n_repeat))
{
    # choose a random sample from dataset (rows: cells, cols: genes)
    # size of random sample is 10% of dataset by default
    idx.runs <- sample.int(dim.dataset[1], size=testSize)
    # print(idx.runs)
    
    # get the neighbors of sample cells; also attach sample cells as self-neighbor
    env <- cbind(knn[idx.runs, seq_len(k0 - 1)], idx.runs)
    # print(env)

    # perform test for each cell in sample
    cf <- if (adapt && is.imbalanced) new.class.frequency else class.frequency  # global batch props
    # apply chi_batch_test to each row of env matrix; 1 is rows
    p.val.test <- apply(env, 1, chi_batch_test, cf, batch, dof)
    # print(p.val.test)

    is.rejected <- p.val.test < alpha

    # apply the same test but this time batch labels are permuted. 
    # permutations are columns of batch.shuff; 2 is columns
    p.val.test.null <- apply(
        batch.shuff, 2,
        function(x) apply(env, 1, chi_batch_test, class.frequency, x, dof)
    )

    # expected rejection rate under the null for this run
    # for each shuffle compute the fraction of cells with p < alpha, then average across shuffles
    kBET.expected[i] <- mean(
        apply(
            p.val.test.null, 2,
            function(x) sum(x < alpha, na.rm = TRUE) / sum(!is.na(x))
        )
    )

    # observed rejection rate in this run
    # fraction of tested cells rejecting the null with the real labels
    kBET.observed[i] <- sum(is.rejected, na.rm = TRUE) / sum(!is.na(p.val.test))

    # compute significance
    kBET.signif[i] <- 1 - ptnorm(
        kBET.observed[i],
        mu = kBET.expected[i],
        sd = sqrt(kBET.expected[i] * (1 - kBET.expected[i]) / testSize),
        alpha = alpha
    )

    # mark which cells were sampled in this run and store
    ## their observed per-cell p-values
    ## their mean null per-cell p-values (averaged over shuffles)
    rejection$results$tested[idx.runs] <- 1
    rejection$results$kBET.pvalue.test[idx.runs] <- p.val.test
    rejection$results$kBET.pvalue.null[idx.runs] <- rowMeans(p.val.test.null, na.rm=TRUE)
}

In [39]:
rejection

kBET.expected,kBET.observed,kBET.signif
<dbl>,<dbl>,<dbl>
0,0,0
0,0,0
0,0,0
0,0,0
tested,kBET.pvalue.test,kBET.pvalue.null
<dbl>,<dbl>,<dbl>
1,0,0.7000649
1,0,0.7717282
1,0,0.4900741


In [40]:
kBET.observed

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [75] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [41]:
length(kBET.observed)

[1] 100

In [42]:
sc_dir

[1] "2/original_kbet/"

In [43]:
# summarize exptected, observed and run-level significance across runs
if (n_repeat > 1)
{
    CI95 <- c(0.025, 0.5, 0.975)

    rejection$summary$kBET.expected <- c(
        mean(kBET.expected, na.rm=TRUE),
        quantile(kBET.expected, CI95, na.rm=TRUE)
    )

    rownames(rejection$summary) <- c('mean', '2.5%', '50%', '97.5%')

    rejection$summary$kBET.observed <- c(
        mean(kBET.observed, na.rm=TRUE),
        quantile(kBET.observed, CI95, na.rm=TRUE)
    )

    rejection$summary$kBET.signif <- c(
        mean(kBET.signif, na.rm=TRUE),
        quantile(kBET.signif, CI95, na.rm=TRUE)
    )

    # also return per run vectors
    rejection$stats$kBET.expected <- kBET.expected
    rejection$stats$kBET.observed <- kBET.observed
    rejection$stats$kBET.signif <- kBET.signif

    if (n_repeat < 10)
    {
        cat('Warning: The quantile computation for ')
        cat(paste0(n_repeat))
        cat(' subset results is not meaningful.')
    }

    if (plot)
    {
        plot.data <- data.frame(
            class = rep(c('observed(kBET)', 'expected(random)'), each=n_repeat),
            data = c(kBET.observed, kBET.expected)
        )

        g <- ggplot(plot.data, aes(class, data)) + geom_boxplot() + theme_bw() + labs(x = 'Test', y = 'Rejection rate') + scale_y_continuous(limits = c(0, 1))
        # print(g)
        ggsave(paste0(sc_dir, "kBET_boxplot.pdf"), plot = g, width = 7, height = 5, dpi = 300)
    }
} else
{
    rejection$summary$kBET.expected <- kBET.expected[1]
    rejection$summary$kBET.observed <- kBET.observed[1]
    rejection$summary$kBET.signif <- kBET.signif[1]
}

In [44]:
# collect parameters
rejection$params <- list()
rejection$params$k0 <- k0
rejection$params$testSize <- testSize
rejection$params$do.pca <- do.pca
rejection$params$dim.pca <- dim.pca
rejection$params$heuristic <- heuristic
rejection$params$n_repeat <- n_repeat
rejection$params$alpha <- alpha
rejection$params$addTest <- addTest
rejection$params$verbose <- verbose
rejection$params$plot <- plot

In [45]:
rejection

$summary
      kBET.expected kBET.observed kBET.signif
mean    0.024777778             1           0
2.5%    0.005555556             1           0
50%     0.022222222             1           0
97.5%   0.044444444             1           0

$results
    tested kBET.pvalue.test kBET.pvalue.null
1        1     0.000000e+00       0.70006488
2        1     0.000000e+00       0.77172817
3        1     0.000000e+00       0.49007412
4        1     0.000000e+00       0.77172817
5        1     0.000000e+00       0.92169573
6        1     0.000000e+00       0.51572741
7        1     0.000000e+00       0.63340070
8        1     0.000000e+00       0.62176061
9        1     0.000000e+00       0.70006488
10       1     0.000000e+00       0.53400848
11       1     0.000000e+00       0.64004168
12       1     0.000000e+00       0.63340070
13       1     0.000000e+00       0.69342390
14       1     0.000000e+00       0.58739070
15       1     0.000000e+00       0.69342390
16       1     0.000000e+00       0.43005091
17       1     0.000000e+00       0.56173741
18       1     0.000000e+00       0.77172817
19       1     0.000000e+00       0.92169573
20       1     0.000000e+00       0.56173741
21       1     0.000000e+00       0.66569497
22       1     0.000000e+00       0.55009732
23       1     0.000000e+00       0.49007412
24       1     0.000000e+00       0.55009732
25       1     0.000000e+00       0.51572741
26       1     0.000000e+00       0.59403168
27       1     0.000000e+00       1.00000000
28       1     0.000000e+00       0.62176061
29       1     0.000000e+00       0.77172817
30       1     0.000000e+00       0.69342390
31       1     0.000000e+00       0.56173741
32       1     0.000000e+00       0.32441717
33       1     0.000000e+00       0.77172817
34       1     0.000000e+00       0.76508719
35       1     0.000000e+00       0.48345026
36       1     0.000000e+00       0.77172817
37       1     0.000000e+00       0.56173741
38       1     0.000000e+00       0.77172817
39       1     0.000000e+00       0.62176061
40       1     0.000000e+00       0.62176061
41       1     0.000000e+00       0.84339146
42       1     0.000000e+00       0.58001847
43       1     0.000000e+00       0.84339146
44       1     0.000000e+00       0.37741706
45       1     0.000000e+00       0.92169573
46       1     0.000000e+00       0.42342706
47       1     0.000000e+00       0.71172209
48       1     0.000000e+00       0.43005091
49       1     0.000000e+00       1.00000000
50       1     0.000000e+00       0.58001847
51       1     0.000000e+00       0.50171421
52       1     0.000000e+00       0.27101782
53       1     0.000000e+00       0.49007412
54       1     0.000000e+00       0.43005091
55       1     0.000000e+00       0.63340070
56       1     0.000000e+00       0.32401772
57       1     0.000000e+00       0.76508719
58       1     0.000000e+00       0.56173741
59       1     0.000000e+00       0.63340070
60       1     0.000000e+00       0.62176061
61       1     0.000000e+00       0.85003244
62       1     0.000000e+00       0.56173741
63       1     0.000000e+00       0.58739070
64       1     0.000000e+00       0.56175453
65       1     0.000000e+00       0.69342390
66       1     0.000000e+00       0.84339146
67       1     0.000000e+00       1.00000000
68       1     0.000000e+00       0.70006488
69       1     0.000000e+00       0.69342390
70       1     0.000000e+00       0.77172817
71       1     0.000000e+00       0.37002771
72       1     0.000000e+00       0.56173741
73       1     0.000000e+00       0.77172817
74       1     0.000000e+00       0.56173741
75       1     0.000000e+00       0.71170497
76       1     0.000000e+00       0.69342390
77       1     0.000000e+00       0.27347662
78       1     0.000000e+00       0.84339146
79       1     0.000000e+00       0.69342390
80       1     0.000000e+00       0.69342390
81       1     0.000000e+00       0.70006488
82       1     0.000000e+00       0.50171421
83       1     0

In [46]:
rejection$summary$kBET.observed

[1] 1 1 1 1

In [47]:
score <- rejection$summary$kBET.observed[1]

In [48]:
score

[1] 1

In [49]:
print("End of the notebook")

[1] "End of the notebook"
